In [ ]:
import os
import tempfile
import time
from os import path

import pandas as pd
import tstore
from tstore.tslong.pandas import open_tslong, to_tstore

# Key Concepts

Let us consider the following dataset of observations from a set of meteorological stations obtained from the Netatmo Weather API (only non-commercial uses of this data are acceptable, see [the Netatmo APIs terms of use](https://dev.netatmo.com/legal)):

In [ ]:
# ts_df_filepath = "../data/interim/netatmo-lausanne-aug-21/ts-df.csv"
ts_df_filepath = "../data/interim/netatmo-lausanne-aug-21/ts-df-clean.csv"

The time series file contains the following columns:
- `variable`: the variable measured (e.g., temperature, humidity...)
- `id`: the identifier of the station
- `time`: the time of the measurement
- `value`: the value of the measurement

In [ ]:
df = pd.read_csv(ts_df_filepath)
df["time"] = pd.to_datetime(df["time"])
df

,variable,id,time,value
0,humidity,70:ee:50:00:10:f0,2021-08-12 13:00:00,50.0
1,humidity,70:ee:50:00:10:f0,2021-08-12 14:00:00,49.0
2,humidity,70:ee:50:00:10:f0,2021-08-12 15:00:00,50.0
3,humidity,70:ee:50:00:10:f0,2021-08-12 16:00:00,49.0
4,humidity,70:ee:50:00:10:f0,2021-08-12 17:00:00,53.0
...,...,...,...,...
381071,wind_strength,70:ee:50:7f:98:de,2021-08-23 22:00:00,2.0
381072,wind_strength,70:ee:50:7f:98:de,2021-08-25 01:00:00,3.0
381073,wind_strength,70:ee:50:7f:98:de,2021-08-25 02:00:00,3.0
381074,wind_strength,70:ee:50:7f:98:de,2021-08-25 03:00:00,2.0


## Time series

A time series is a sequence of data indexed by timestamps. In our case, we have multiple time series, each corresponding to a meteorological variable measured by a station module. Consider data for the following station

In [ ]:
station_id = "70:ee:50:00:10:f0"
df.loc[df["id"] == station_id]

,variable,id,time,value
0,humidity,70:ee:50:00:10:f0,2021-08-12 13:00:00,50.0
1,humidity,70:ee:50:00:10:f0,2021-08-12 14:00:00,49.0
2,humidity,70:ee:50:00:10:f0,2021-08-12 15:00:00,50.0
3,humidity,70:ee:50:00:10:f0,2021-08-12 16:00:00,49.0
4,humidity,70:ee:50:00:10:f0,2021-08-12 17:00:00,53.0
...,...,...,...,...
234565,temperature,70:ee:50:00:10:f0,2021-08-25 01:00:00,11.4
234566,temperature,70:ee:50:00:10:f0,2021-08-25 02:00:00,10.9
234567,temperature,70:ee:50:00:10:f0,2021-08-25 05:00:00,11.4
234568,temperature,70:ee:50:00:10:f0,2021-08-25 06:00:00,13.0


In order to have a proper time series data structure, we can pivot the data to have each variable as a column and each observation as a row (see more details in the [data forms section below](#data-forms)):

In [ ]:
ts_df = df.loc[df["id"] == station_id].pivot_table(
    index="time",
    columns="variable",
    values="value",
)
ts_df

variable,humidity,pressure,temperature
time,,,
2021-08-12 13:00:00,50.0,1023.7,28.1
2021-08-12 14:00:00,49.0,1023.3,29.5
2021-08-12 15:00:00,50.0,1023.3,29.4
2021-08-12 16:00:00,49.0,1022.2,29.2
2021-08-12 17:00:00,53.0,1023.3,27.9
...,...,...,...
2021-08-25 01:00:00,80.0,1019.3,11.4
2021-08-25 02:00:00,81.0,1019.2,10.9
2021-08-25 05:00:00,83.0,1019.2,11.4


The time series above can be encapsulated in a `TS` object:

In [ ]:
ts = tstore.TS(ts_df)
ts

TS[start=2021-08-12 13:00:00,end=2021-08-25 07:00:00;shape=(247, 3)]

Why would we need such an encapsulation? Let us get the data for two stations and pivot it to have each variable *for each station* as a column and each observation as a row:

In [ ]:
station_ids = ["70:ee:50:00:c7:72", "70:ee:50:15:ea:9e"]
ts_df = df[df["id"].isin(station_ids)].pivot_table(
    index="time",
    columns=["id", "variable"],
    values="value",
)
ts_df

id                  70:ee:50:00:c7:72 70:ee:50:15:ea:9e 70:ee:50:00:c7:72  \
variable                     humidity          humidity          pressure   
time                                                                        
2021-08-12 12:00:00               NaN              49.0               NaN   
2021-08-12 13:00:00               NaN              52.0               NaN   
2021-08-12 14:00:00              53.0              51.0            1024.3   
2021-08-12 15:00:00              51.0              54.0            1023.9   
2021-08-12 16:00:00              53.0              52.0            1023.1   
...                               ...               ...               ...   
2021-08-25 05:00:00               NaN              90.0               NaN   
2021-08-25 06:00:00               NaN              90.0               NaN   
2021-08-25 07:00:00               NaN              90.0               NaN   
2021-08-25 08:00:00              77.0              89.0            1021.8   
2021-08-25 09:00:00              76.0              86.0            1021.9   

id                  70:ee:50:15:ea:9e           70:ee:50:00:c7:72  \
variable                     pressure rain_live       temperature   
time                                                                
2021-08-12 12:00:00            1019.8       0.0               NaN   
2021-08-12 13:00:00            1019.6       0.0               NaN   
2021-08-12 14:00:00            1019.6       0.0              30.0   
2021-08-12 15:00:00            1019.2       0.0              30.2   
2021-08-12 16:00:00            1018.3       0.0              30.2   
...                               ...       ...               ...   
2021-08-25 05:00:00            1015.5       0.0               NaN   
2021-08-25 06:00:00            1015.7       0.0               NaN   
2021-08-25 07:00:00            1015.7       0.0               NaN   
2021-08-25 08:00:00            1016.0       0.0              15.4   
2021-08-25 09:00:00            1016.1       0.0              16.2   

id                  70:ee:50:15:ea:9e  
variable                  temperature  
time                                   
2021-08-12 12:00:00              31.6  
2021-08-12 13:00:00              29.9  
2021-08-12 14:00:00              29.4  
2021-08-12 15:00:00              29.7  
2021-08-12 16:00:00              30.4  
...                               ...  
2021-08-25 05:00:00              11.5  
2021-08-25 06:00:00              11.9  
2021-08-25 07:00:00              12.4  
2021-08-25 08:00:00              13.1  
2021-08-25 09:00:00              14.0  

[223 rows x 7 columns]

Note now that each station has several missing values, therefore aligning the time index would result in many nan values:

In [ ]:
ts_df.isna().sum(axis="rows")

id                 variable   
70:ee:50:00:c7:72  humidity        45
70:ee:50:15:ea:9e  humidity       105
70:ee:50:00:c7:72  pressure        45
70:ee:50:15:ea:9e  pressure       105
                   rain_live      105
70:ee:50:00:c7:72  temperature     45
70:ee:50:15:ea:9e  temperature    105
dtype: int64

Instead, we can create a `TS` object for each station and put them in a pandas object. Let us first transform it into a long data frame:

In [ ]:
long_ts_df = df[df["id"].isin(station_ids)].pivot_table(
    index=["id", "time"],
    columns="variable",
    values="value",
)
long_ts_df

variable                               humidity  pressure  rain_live  \
id                time                                                 
70:ee:50:00:c7:72 2021-08-12 14:00:00      53.0    1024.3        NaN   
                  2021-08-12 15:00:00      51.0    1023.9        NaN   
                  2021-08-12 16:00:00      53.0    1023.1        NaN   
                  2021-08-12 17:00:00      55.0    1023.7        NaN   
                  2021-08-12 18:00:00      61.0    1023.5        NaN   
...                                         ...       ...        ...   
70:ee:50:15:ea:9e 2021-08-25 05:00:00      90.0    1015.5        0.0   
                  2021-08-25 06:00:00      90.0    1015.7        0.0   
                  2021-08-25 07:00:00      90.0    1015.7        0.0   
                  2021-08-25 08:00:00      89.0    1016.0        0.0   
                  2021-08-25 09:00:00      86.0    1016.1        0.0   

variable                               temperature  
id                time                              
70:ee:50:00:c7:72 2021-08-12 14:00:00         30.0  
                  2021-08-12 15:00:00         30.2  
                  2021-08-12 16:00:00         30.2  
                  2021-08-12 17:00:00         29.0  
                  2021-08-12 18:00:00         27.4  
...                                            ...  
70:ee:50:15:ea:9e 2021-08-25 05:00:00         11.5  
                  2021-08-25 06:00:00         11.9  
                  2021-08-25 07:00:00         12.4  
                  2021-08-25 08:00:00         13.1  
                  2021-08-25 09:00:00         14.0  

[296 rows x 4 columns]

We can now transform the above data frame into a series of `TS` objects:

In [ ]:
ts_ser = pd.Series(
    tstore.TSArray(
        [tstore.TS(long_ts_df.loc[station_id]) for station_id in station_ids],
    ),
    index=station_ids,
)
ts_ser

70:ee:50:00:c7:72    TS[start=2021-08-12 14:00:00,end=2021-08-25 09...
70:ee:50:15:ea:9e    TS[start=2021-08-12 12:00:00,end=2021-08-25 09...
dtype: TS[dask.Series]

Note that a `TSArray` is [a pandas `ExtensionArray` which is used to support custom data types](https://pandas.pydata.org/docs/reference/api/pandas.api.extensions.ExtensionArray.html), in this case, `TS` objects.

## TSDF

In some cases columns with different time indexing may share the same identifier. For instance, Netatmo stations can have up to three modules: one for temperature, humidity and pressure, another for wind and a last one for rain. Note that even for the same station, each of this modules can have its own time indexing.

In [ ]:
# station_ids = ["70:ee:50:00:3b:86", "70:ee:50:00:53:26"]
station_ids = long_ts_df.index.get_level_values("id").unique()
long_ts_df = df.pivot_table(index=["id", "time"], columns="variable", values="value")
# TODO: TS objects of nan only should be nan values
ts_df = pd.DataFrame(
    {
        variable: tstore.TSArray(
            [
                tstore.TS(long_ts_df.loc[station_id][variable].dropna())
                for station_id in station_ids
            ],
        )
        for variable in long_ts_df.columns
    },
    index=station_ids,
)
ts_df

,humidity,pressure,rain_live,temperature,wind_angle,wind_strength
id,,,,,,
70:ee:50:00:10:f0,"TS[shape=(247,),start=2021-08-12 13:00:00,end=...","TS[shape=(247,),start=2021-08-12 13:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(247,),start=2021-08-12 13:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"
70:ee:50:00:17:8e,"TS[shape=(41,),start=2021-08-17 00:00:00,end=2...","TS[shape=(41,),start=2021-08-17 00:00:00,end=2...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(41,),start=2021-08-17 00:00:00,end=2...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"
70:ee:50:00:20:88,"TS[shape=(295,),start=2021-08-12 12:00:00,end=...","TS[shape=(295,),start=2021-08-12 12:00:00,end=...","TS[shape=(295,),start=2021-08-12 12:00:00,end=...","TS[shape=(295,),start=2021-08-12 12:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"
70:ee:50:00:30:2a,"TS[shape=(246,),start=2021-08-12 12:00:00,end=...","TS[shape=(246,),start=2021-08-12 12:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(246,),start=2021-08-12 12:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"
70:ee:50:00:36:d0,"TS[shape=(178,),start=2021-08-12 13:00:00,end=...","TS[shape=(178,),start=2021-08-12 13:00:00,end=...","TS[shape=(178,),start=2021-08-12 13:00:00,end=...","TS[shape=(178,),start=2021-08-12 13:00:00,end=...","TS[shape=(178,),start=2021-08-12 13:00:00,end=...","TS[shape=(178,),start=2021-08-12 13:00:00,end=..."
...,...,...,...,...,...,...
70:ee:50:7a:c4:fc,"TS[shape=(155,),start=2021-08-12 22:00:00,end=...","TS[shape=(155,),start=2021-08-12 22:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(155,),start=2021-08-12 22:00:00,end=...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"
70:ee:50:7a:d2:c2,"TS[shape=(88,),start=2021-08-12 13:00:00,end=2...","TS[shape=(88,),start=2021-08-12 13:00:00,end=2...","TS[shape=(88,),start=2021-08-12 13:00:00,end=2...","TS[shape=(88,),start=2021-08-12 13:00:00,end=2...","TS[shape=(88,),start=2021-08-12 13:00:00,end=2...","TS[shape=(88,),start=2021-08-12 13:00:00,end=2..."
70:ee:50:7a:dc:ce,"TS[shape=(12,),start=2021-08-15 14:00:00,end=2...","TS[shape=(12,),start=2021-08-15 14:00:00,end=2...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(12,),start=2021-08-15 14:00:00,end=2...","TS[shape=(0,),start=NaT,end=NaT]","TS[shape=(0,),start=NaT,end=NaT]"


Note that we have many nan values. Finally, it is also possible to have a single `TS` object per module and station, in which case the `TS` objects of the first and third module would have multiple variables, all of which would be aligned in time:

In [ ]:
module_dict = {
    "module_1": ["temperature", "humidity", "pressure"],
    "module_2": ["rain_live"],
    "module_3": ["wind_angle", "wind_strength"],
}
ts_df = pd.DataFrame(
    {
        module: tstore.TSArray(
            [
                tstore.TS(long_ts_df.loc[station_id][variables])
                for station_id in station_ids
            ],
        )
        for module, variables in module_dict.items()
    },
    index=station_ids,
)
ts_df

,module_1,module_2,module_3
id,,,
70:ee:50:00:10:f0,"TS[shape=(247, 3),start=2021-08-12 13:00:00,en...","TS[shape=(247, 1),start=2021-08-12 13:00:00,en...","TS[shape=(247, 2),start=2021-08-12 13:00:00,en..."
70:ee:50:00:17:8e,"TS[shape=(41, 3),start=2021-08-17 00:00:00,end...","TS[shape=(41, 1),start=2021-08-17 00:00:00,end...","TS[shape=(41, 2),start=2021-08-17 00:00:00,end..."
70:ee:50:00:20:88,"TS[shape=(295, 3),start=2021-08-12 12:00:00,en...","TS[shape=(295, 1),start=2021-08-12 12:00:00,en...","TS[shape=(295, 2),start=2021-08-12 12:00:00,en..."
70:ee:50:00:30:2a,"TS[shape=(246, 3),start=2021-08-12 12:00:00,en...","TS[shape=(246, 1),start=2021-08-12 12:00:00,en...","TS[shape=(246, 2),start=2021-08-12 12:00:00,en..."
70:ee:50:00:36:d0,"TS[shape=(178, 3),start=2021-08-12 13:00:00,en...","TS[shape=(178, 1),start=2021-08-12 13:00:00,en...","TS[shape=(178, 2),start=2021-08-12 13:00:00,en..."
...,...,...,...
70:ee:50:7a:c4:fc,"TS[shape=(155, 3),start=2021-08-12 22:00:00,en...","TS[shape=(155, 1),start=2021-08-12 22:00:00,en...","TS[shape=(155, 2),start=2021-08-12 22:00:00,en..."
70:ee:50:7a:d2:c2,"TS[shape=(88, 3),start=2021-08-12 13:00:00,end...","TS[shape=(88, 1),start=2021-08-12 13:00:00,end...","TS[shape=(88, 2),start=2021-08-12 13:00:00,end..."
70:ee:50:7a:dc:ce,"TS[shape=(12, 3),start=2021-08-15 14:00:00,end...","TS[shape=(12, 1),start=2021-08-15 14:00:00,end...","TS[shape=(12, 2),start=2021-08-15 14:00:00,end..."


TODO: geospatial example

<a id='data-forms'></a>
## Data forms

Building upon [the documentation of seaborn](https://seaborn.pydata.org/tutorial/data_structure.html), we can define the following data forms:
- **Long form**, where each variable is a column and each observation is a row.
- **Wide form**, where columns and rows contain levels of a variable. Each observation is defined by its value at the cell and its coordinates in both the row and column indices.

However, eventhough our dataset clearly looks more similar to the long form, it is not exactly a long form dataset since we do not have each variable as a column but instead have multiple variables in the same column. This kind of dataset can be colled "messy" or "untidy" data (see [the work on "data tyding"](https://vita.had.co.nz/papers/tidy-data.pdf) by Hadley Wickham), where each row corresponds to the unit of data collection (here, the meteorological measurement). As shown above, we can convert this dataset to a long form using [`pandas.DataFrame.pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html).

In [ ]:
long_ts_df = df.pivot_table(
    index=["id", "time"],
    columns="variable",
    values="value",
).reset_index()
long_ts_df

variable,id,time,humidity,pressure,rain_live,temperature,wind_angle,wind_strength
0,70:ee:50:00:10:f0,2021-08-12 13:00:00,50.0,1023.7,NaN,28.1,NaN,NaN
1,70:ee:50:00:10:f0,2021-08-12 14:00:00,49.0,1023.3,NaN,29.5,NaN,NaN
2,70:ee:50:00:10:f0,2021-08-12 15:00:00,50.0,1023.3,NaN,29.4,NaN,NaN
3,70:ee:50:00:10:f0,2021-08-12 16:00:00,49.0,1022.2,NaN,29.2,NaN,NaN
4,70:ee:50:00:10:f0,2021-08-12 17:00:00,53.0,1023.3,NaN,27.9,NaN,NaN
...,...,...,...,...,...,...,...,...
101454,70:ee:50:7f:98:de,2021-08-23 22:00:00,64.0,1019.8,0.0,19.0,52.0,2.0
101455,70:ee:50:7f:98:de,2021-08-25 01:00:00,69.0,1018.3,0.0,17.5,52.0,3.0
101456,70:ee:50:7f:98:de,2021-08-25 02:00:00,70.0,1018.1,0.0,17.2,60.0,3.0
101457,70:ee:50:7f:98:de,2021-08-25 03:00:00,71.0,1018.2,0.0,16.8,54.0,2.0


## TStore

In [ ]:
def tstore_tree(base_dir, head=10, tail=10):
    result = [
        path.join(dp, f) for dp, dn, filenames in os.walk(base_dir) for f in filenames
    ]
    return result[:head] + ["..."] + result[-tail:]


# try round-trip
# base_dir = "/tmp/dummy_tstore"
tmp_dir = "/tmp/dummy_tstore"
tstore_structure = "id-var"
overwrite = True
id_var = "id"
time_var = "time"
# static_variables = ["variable"]
geometry = None  # NOT IMPLEMENTED YET

# Same partitioning for all TS
partitioning = "day"

# Each timeseries is a TS object
# TODO: how do I put humidity/temperature in the same TS object?
# the line below does not work
ts_variables = ["humidity", "temperature"]
# ts_variables = {var: ["id", "value"] for var in ts_df["variable"].unique()}
# Group multiple timeseries into one TS object
# ts_variables = {"precipitation": ["name", "id", "x", "y"]}

with tempfile.TemporaryDirectory() as _tmp_dir:
    start_write = time.time()
    to_tstore(
        long_ts_df,
        # TSTORE options
        tmp_dir,
        # DFLONG attributes
        id_var=id_var,
        time_var=time_var,
        ts_variables=ts_variables,
        # static_variables=static_variables,
        # TSTORE options
        partitioning=partitioning,
        tstore_structure=tstore_structure,
        overwrite=overwrite,
    )
    end_write = time.time()
    for line in tstore_tree(tmp_dir):
        print(line)
    print("`to_tstore` time (s):", end_write - start_write)

    start_read = time.time()
    df = open_tslong(tmp_dir, ts_variables=list(ts_variables))
    print("`open_tslong` time (s):", time.time() - start_read)

/tmp/dummy_tstore/tstore_metadata.yml
/tmp/dummy_tstore/_attributes.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/_common_metadata
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/_metadata
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=21/part-0.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=15/part-0.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=13/part-0.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=18/part-0.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=24/part-0.parquet
/tmp/dummy_tstore/70:ee:50:2c:95:e4/temperature/day=19/part-0.parquet
...
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidity/day=18/part-0.parquet
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidity/day=24/part-0.parquet
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidity/day=20/part-0.parquet
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidity/day=19/part-0.parquet
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidity/day=22/part-0.parquet
/tmp/dummy_tstore/70:ee:50:3c:28:cc/humidi